In [1]:
# third-party
import pandas as pd
import scipy
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot, get_relative_errors
from get_data import get_data_by_id_activity
from constants import CATEGORICAL_PALETTE


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

In [3]:
acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)


Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [17]:
def plot_resp_param(data, overview, id, activity, sensor, sampling_freq=100):

    sensor_dict = {"MAG": "mag", "PZT": "pzt"}

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # plot sensor and airflow signals
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity][sensor_dict[sensor]]),len(data[id][activity][sensor_dict[sensor]]))/sampling_freq, 
                             y=data[id][activity][sensor_dict[sensor]], 
                             name=sensor))
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity]['airflow']),len(data[id][activity]['airflow']))/sampling_freq,
                             y=data[id][activity]['airflow'], 
                            name="Airflow", line={"color":"white"}), secondary_y=True)

    # add FR events
    peak_valley = {"TP_i": "valleys", "TP_e": "peaks", "FN_i": "valleys", "FN_e": "peaks"}

    for event in ["TP_i", "TP_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"TP {peak_valley[event]}"))
        
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity]["Airflow"][peak_valley[event]])/sampling_freq, y=data[id][activity]['airflow'][overview[id][activity]["Airflow"][peak_valley[event]]],
                        mode="markers",
                        name=f"ref {peak_valley[event]}", marker={"color":"white"}), secondary_y=True)
    
    for event in ["FN_i", "FN_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"FN {peak_valley[event]}"))


    TP_peaks_tuple = [(event, True, "peak") for event in overview[id][activity][sensor]["TP_e"]]
    TP_valleys_tuple = [(event, True, "valley") for event in overview[id][activity][sensor]["TP_i"]]
    FN_peaks_tuple = [(event, False, "peak") for event in overview[id][activity][sensor]["FN_e"]]
    FN_valleys_tuple = [(event, False, "valley") for event in overview[id][activity][sensor]["FN_i"]]
    extrema = sorted(TP_peaks_tuple + TP_valleys_tuple + FN_peaks_tuple + FN_valleys_tuple)

    
    extrema_pairs_peaks = np.asarray([
        (extrema[i][0], extrema[i+1][0],
            all((extrema[i][1], extrema[i+1][1])))
        for i in range(0, len(extrema)-1)
        if extrema[i][2] == "peak" and extrema[i+1][2] == "valley"
    ])

    extrema_pairs_valleys = np.asarray([
        (extrema[i][0], extrema[i+1][0],
            all((extrema[i][1], extrema[i+1][1])))
        for i in range(0, len(extrema)-1)
        if extrema[i][2] == "valley" and extrema[i+1][2] == "peak"
    ])

    extrema_pairs_peaks = extrema_pairs_peaks[extrema_pairs_peaks[:,-1] != 0]
    extrema_pairs_valleys = extrema_pairs_valleys[extrema_pairs_valleys[:,-1] != 0]

    for i,ind in enumerate(extrema_pairs_peaks):
        try:
            exp_duration = overview[id][activity][sensor]["tE (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+exp_duration, fillcolor=CATEGORICAL_PALETTE[0], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="exp", annotation_position="top left",)
        except Exception as e:
            print(e)

    for i, ind in enumerate(extrema_pairs_valleys):  
        try:
            insp_duration = overview[id][activity][sensor]["tI (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+insp_duration, fillcolor=CATEGORICAL_PALETTE[1], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="insp", annotation_position="bottom left",)
        except Exception as e:
            print(e)
                
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

    print(f'sensor: {overview[id][activity][sensor]["tI (s)"]} | airflow: {overview[id][activity][sensor]["tI airflow (s)"]}')
    

In [24]:

plot_resp_param(data, overview, id="1BST", activity="UAL", sensor="MAG")


sensor: [1.26 1.74 1.75 1.59 1.62 0.7  1.44 1.62 1.62 1.59 1.18 1.55] | airflow: [2.28 1.67 2.3  2.52 1.76 1.58 1.86 1.66 1.8  1.51 1.69 1.64]


In [22]:
plot_resp_param(data, overview, id="Y6O3", activity="SGB", sensor="PZT")

sensor: [1.74 2.08 5.87 4.85 1.44] | airflow: [ 4.06  3.31 13.75  3.74  3.57]


### Activity-specific only RP analysis

In [7]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['STNB', 'SNB', 'SGB', 'MIXB', 'TR', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS'])

In [8]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,STNB,MAG,0.29 $\pm$ 0.47,12.41 $\pm$ 16.60,0.33 $\pm$ 0.55,10.02 $\pm$ 20.40,0.22 $\pm$ 0.43,4.56 $\pm$ 7.89
1,STNB,PZT,0.57 $\pm$ 0.53,26.07 $\pm$ 19.80,0.63 $\pm$ 0.60,17.62 $\pm$ 36.13,0.32 $\pm$ 0.40,7.01 $\pm$ 8.97
2,SNB,MAG,0.30 $\pm$ 0.46,14.09 $\pm$ 19.58,0.30 $\pm$ 0.45,11.36 $\pm$ 14.43,0.20 $\pm$ 0.33,4.31 $\pm$ 6.95
3,SNB,PZT,0.56 $\pm$ 0.59,26.62 $\pm$ 27.50,0.56 $\pm$ 0.57,20.42 $\pm$ 22.36,0.40 $\pm$ 0.52,8.78 $\pm$ 10.41
4,SGB,MAG,1.08 $\pm$ 0.88,25.85 $\pm$ 22.45,1.15 $\pm$ 0.84,21.17 $\pm$ 18.40,0.81 $\pm$ 0.70,8.05 $\pm$ 7.09
5,SGB,PZT,1.46 $\pm$ 1.24,32.91 $\pm$ 22.47,1.83 $\pm$ 1.61,21.34 $\pm$ 46.62,1.09 $\pm$ 1.25,10.94 $\pm$ 12.92
6,MIXB,MAG,0.35 $\pm$ 0.44,14.33 $\pm$ 16.50,0.35 $\pm$ 0.46,11.69 $\pm$ 13.70,0.26 $\pm$ 0.39,4.51 $\pm$ 5.99
7,MIXB,PZT,0.79 $\pm$ 0.74,24.63 $\pm$ 38.07,0.81 $\pm$ 0.94,25.10 $\pm$ 20.20,0.54 $\pm$ 0.79,9.09 $\pm$ 10.89
8,TR,MAG,0.63 $\pm$ 0.55,30.23 $\pm$ 25.16,0.66 $\pm$ 0.60,27.60 $\pm$ 30.47,0.56 $\pm$ 0.54,12.92 $\pm$ 12.87
9,TR,PZT,0.72 $\pm$ 0.65,34.11 $\pm$ 28.12,0.75 $\pm$ 0.65,32.76 $\pm$ 24.60,0.64 $\pm$ 0.67,14.01 $\pm$ 12.88


### Overall RP analysis

In [9]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [10]:
get_breath_parameters_performance(overview_all, target=None)

N=3360 tI for MAG
N=3296 tI for PZT
N=3358 tE for MAG
N=3282 tE for PZT
N=3264 tB for MAG
N=3172 tB for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%),Slope,Intercept,R^2
0,tI,MAG,0.38 $\pm$ 0.43,19.34 $\pm$ 18.75,1.00,0.05,0.65
1,tI,PZT,0.63 $\pm$ 0.59,30.28 $\pm$ 27.55,0.77,0.47,0.37
2,tE,MAG,0.39 $\pm$ 0.44,17.55 $\pm$ 20.22,0.84,0.28,0.70
3,tE,PZT,0.65 $\pm$ 0.65,27.57 $\pm$ 29.77,0.86,0.29,0.51
4,tB,MAG,0.31 $\pm$ 0.41,7.37 $\pm$ 9.11,0.98,0.06,0.91
5,tB,PZT,0.51 $\pm$ 0.61,12.31 $\pm$ 13.36,0.97,0.09,0.80


In [11]:
import plotly.graph_objects as go

for metric in ["tI", "tE", "tB"]:
    rel_error_mag = get_relative_errors(overview_all["MAG"][f"{metric} airflow (s)"], overview_all["MAG"][f"{metric} (s)"])
    rel_error_pzt = get_relative_errors(overview_all["PZT"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"])

    print(f"{metric}: {scipy.stats.mannwhitneyu(rel_error_mag, rel_error_pzt)}")

    fig = go.Figure()
    fig.add_trace(go.Violin(y=rel_error_mag, name="MAG"))
    fig.add_trace(go.Violin(y=rel_error_pzt, name="PZT"))

    # fig.show()

    # normality_test(rel_error_mag, sensor="MAG", type=f"relative error {metric}")
    # normality_test(rel_error_pzt, sensor="PZT", type=f"relative error {metric}")


tI: MannwhitneyuResult(statistic=5283842.0, pvalue=0.0012232658432534498)
tE: MannwhitneyuResult(statistic=5377860.0, pvalue=0.08948744974487069)
tB: MannwhitneyuResult(statistic=5103184.0, pvalue=0.32387262635288616)


### Bias and variability analysis by activity

In [12]:
for id in overview.keys():
    for activity in overview[id].keys():
        for sensor in ["MAG", "PZT"]:
            if any(overview[id][activity][sensor]["tI airflow (s)"] > 10):
                print(f'{overview[id][activity][sensor]["tI airflow (s)"]} ({id} - {activity} - {sensor})')

[ 4.06  3.31 13.75  3.74  3.57] (Y6O3 - SGB - PZT)


In [13]:
from performance import get_bias_variability

get_bias_variability(overview_all_participants, target="Activity", metric="tI", relative_error=False)

,Activity,Sensor,Bias (s),Variability (s)
0,STNB,MAG,0.022,0.548
1,STNB,PZT,0.270,0.737
2,SNB,MAG,0.126,0.535
3,SNB,PZT,0.291,0.765
4,SGB,MAG,0.366,1.351
5,SGB,PZT,-0.474,1.861
6,MIXB,MAG,0.181,0.529
7,MIXB,PZT,0.311,1.040
8,TR,MAG,0.177,0.823
9,TR,PZT,0.139,0.959


In [14]:
activity = "UAL"
for metric in ["tI", "tE", "tB"]:
        bland_altman_plot(overview_all["MAG"][f"{metric} (s)"], overview_all["MAG"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"], overview_all["PZT"][f"{metric} airflow (s)"], metric)
        #bland_altman_plot(overview_all_participants[activity]["MAG"][f"{metric} (s)"], overview_all_participants[activity]["MAG"][f"{metric} airflow (s)"], overview_all_participants[activity]["PZT"][f"{metric} (s)"], overview_all_participants[activity]["PZT"][f"{metric} airflow (s)"], metric, activity)


MAG | mean: 0.056, lloa: -1.06, uloa: 1.17 
PZT | mean: 0.015, lloa: -1.68, uloa: 1.71 


MAG | mean: -0.057, lloa: -1.20, uloa: 1.09 
PZT | mean: -0.014, lloa: -1.82, uloa: 1.79 


MAG | mean: -0.005, lloa: -1.01, uloa: 1.00 
PZT | mean: -0.024, lloa: -1.58, uloa: 1.53 
